In [1]:
# --- Step 1: Build the STO structure using ASE ---
from ase import Atoms
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

# For a simple cubic perovskite, we define the lattice constant (in Å)
a = 3.905  
# Here we build a simple representation of STO.
# (In practice, you need to set the positions appropriately;
#  this is a schematic example.)
sto = Atoms(cell=[(a, 0, 0), (0, a, 0), (0, 0, a)],
            scaled_positions=[(0, 0, 0),         # Sr
                              (0.5, 0.5, 0.5),   # Ti
                              (0.5, 0, 0),       # O1
                              (0, 0.5, 0),       # O2
                              (0, 0, 0.5)],      # O3
            pbc=True)
            
# --- Step 2: Ground-State DFT Calculation with GPAW ---
from gpaw import GPAW, PW

# Set up the calculator: adjust parameters as needed (e.g. plane wave cutoff, k-points)
calc = GPAW(mode=PW(400),        # plane-wave cutoff in eV
            xc='PBE',            # exchange-correlation functional
            kpts=(6, 6, 6),      # k-point grid
            txt='sto_gs.txt')    # output log file

sto.set_calculator(calc)
energy = sto.get_potential_energy()
print("Ground state energy (eV):", energy)

# Save the ground-state wavefunctions/density if needed.
calc.write('sto_ground_state.gpw', mode='all')

# --- Step 3: Compute the Frequency-Dependent Dielectric Function ---
# There are two typical approaches:
# A) Using TDDFT / linear response (if available in GPAW)
# B) Using DFPT / phonon calculations (possibly with Phonopy) and then modeling the response
#
# Here is a schematic example for approach (A). Note that GPAW's response module may 
# require additional setup and is under active development. Consult the GPAW documentation.

try:
    from gpaw.response import ResponseCalculator
except ImportError:
    raise ImportError("GPAW's ResponseCalculator module not found. "
                      "Check your GPAW installation and documentation for linear response calculations.")

# Initialize the response calculator using the converged ground-state calculator.
response = ResponseCalculator(calc)

# Define the frequency range in eV.
# Convert 400 cm^-1 to 900 cm^-1 to eV:
#   1 cm^-1 ≈ 1.23984e-4 eV => 400 cm^-1 ≈ 0.0496 eV, 900 cm^-1 ≈ 0.1116 eV
omega_min = 400 * 1.23984e-4  # ~0.0496 eV
omega_max = 900 * 1.23984e-4  # ~0.1116 eV
omega = np.linspace(omega_min, omega_max, 100)  # 100 frequency points

# For each frequency, compute the dielectric function.
# (The actual API for ResponseCalculator may differ. This is illustrative.)
eps_real = []
eps_imag = []
for w in omega:
    # The following function call is illustrative.
    # You may need to adjust it depending on GPAW's current response API.
    eps = response.dielectric_function(omega=w)
    # Assume eps is a complex number (or an array if you compute tensor components)
    eps_real.append(eps.real)
    eps_imag.append(eps.imag)

eps_real = np.array(eps_real)
eps_imag = np.array(eps_imag)

# --- Step 4: Plot the Dielectric Function ---
plt.figure(figsize=(8, 5))
plt.plot(omega, eps_real, label='Re[$\epsilon$]')
plt.plot(omega, eps_imag, label='Im[$\epsilon$]')
plt.xlabel('Energy (eV)')
plt.ylabel('Dielectric Function')
plt.title('Dielectric Function of STO (400–900 cm⁻¹)')
plt.legend()
plt.show()


<>:77: SyntaxWarning: invalid escape sequence '\e'
<>:78: SyntaxWarning: invalid escape sequence '\e'
<>:77: SyntaxWarning: invalid escape sequence '\e'
<>:78: SyntaxWarning: invalid escape sequence '\e'
C:\Users\leona\AppData\Local\Temp\ipykernel_5556\1602104732.py:77: SyntaxWarning: invalid escape sequence '\e'
  plt.plot(omega, eps_real, label='Re[$\epsilon$]')
C:\Users\leona\AppData\Local\Temp\ipykernel_5556\1602104732.py:78: SyntaxWarning: invalid escape sequence '\e'
  plt.plot(omega, eps_imag, label='Im[$\epsilon$]')
C:\Users\leona\AppData\Local\Temp\ipykernel_5556\1602104732.py:77: SyntaxWarning: invalid escape sequence '\e'
  plt.plot(omega, eps_real, label='Re[$\epsilon$]')
C:\Users\leona\AppData\Local\Temp\ipykernel_5556\1602104732.py:78: SyntaxWarning: invalid escape sequence '\e'
  plt.plot(omega, eps_imag, label='Im[$\epsilon$]')


ModuleNotFoundError: No module named 'gpaw'